In [189]:
import pandas as pd
import numpy as np
from statsmodels.tsa.ar_model import AutoReg
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy.stats import norm
import statsmodels.api as sm
from arch.unitroot import ADF
from statsmodels.discrete.discrete_model import Probit

In [79]:
DEFAULT_FIGSIZE = (48,18)
plt.rc("figure", figsize=DEFAULT_FIGSIZE)
plt.rc("font", size=48)

def plot(s, y=DEFAULT_FIGSIZE):
    #figsize(y=y)
    fig, ax = plt.subplots(1, 1)
    if isinstance(s, pd.Series):
        s.plot(ax=ax, legend=False, color=["orange"])
    else:
        s.plot(ax=ax, legend=False)
        fig.legend(frameon=False)
    
    ax.set_xlabel(None)
    ax.set_xlim(s.index[0], s.index[-1]) #s.index[0]
    sns.despine()
    fig.tight_layout(pad=1.0)
    #figsize()

In [40]:
## is a questionable decision to use dm: essentially stating forecast only lost if it lost really really badly

def diebold_mariano(loss_a, loss_b, nw_bandwidth,cv):
    delta = loss_a - loss_b
    mod = sm.OLS(delta, np.ones_like(delta))
    dm_res = mod.fit(cov_type="HAC", cov_kwds={"maxlags":int(nw_bandwidth)})
    av_diff = delta.mean()
    a  = dm_res.bse[0]
    dm_stat = float(av_diff / a)  

    #cv = norm.ppf(0.975)

    if dm_stat < (cv * -1):
        concl = 1
    else:
        concl = 0


    return concl

QUESTION 1
--

In [438]:
# Simulate some data
# run time ~ 1 minute

# True simulated data ## Sample size 100
rg = np.random.RandomState(100)

r = [0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
runs = 1000
true_100 = {}
size = 100
nw = size ** (1/3)
cv = norm.ppf(0.95)
theta_100 = []
dm_100 = []

for rho in r:
    print(rho)
    for run in range(runs):
        start = rg.normal(0,1/(1-(rho**2)))
        run_holder = []
        run_holder.append(start)

        for i in range(1,size):
            shock = rg.standard_normal()
            last_val = run_holder[-1]
            new_val = last_val * rho + shock
            run_holder.append(new_val)

        y = np.asarray(run_holder)

        # Fit using first half, tau//2
        mod = AutoReg(y[:size//2], lags=1, trend="c", old_names=False)
        res = mod.fit()

        # Full-sample model
        oos_mod = AutoReg(y, lags=1, trend="c", old_names=False)

        # One-step predictions
        oos_1step = oos_mod.predict(res.params)

        # OOS Random Walk predictions
        oos_rw = oos_mod.predict([0, 1])

        # Get second half of both
        # Use -tau//2: to get second half
        oos_1step = oos_1step[-size//2:]
        oos_rw = oos_rw[-size//2:]

        # Calculate the losses
        loss_1step = (oos_1step - y[-size//2:]) ** 2
        loss_rw = (oos_rw - y[-size//2:]) ** 2

        evaluation = (loss_1step - loss_rw).sum()

        if evaluation > 0:
            mark = 1
        else:
            mark = 0
        dm_100.append(mark)

        # Append the estimated thetas and the DM results into lists
        theta_100.append(res.params[1])

        #dm_100.append(diebold_mariano( loss_rw, loss_1step, nw, cv))


0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99


In [439]:
# 1. SIMPLE OLS APPROACH

dm_100 = np.asarray(dm_100)
theta_100 = np.asarray(theta_100)
# x = np.asarray(pd.DataFrame([theta_100,(theta_100)**2]).T) ### can be used to get a non-linear relationship?
res = sm.OLS(dm_100, sm.add_constant(theta_100)).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     422.1
Date:                Fri, 12 Mar 2021   Prob (F-statistic):           6.75e-92
Time:                        09:32:45   Log-Likelihood:                -7030.7
No. Observations:               10000   AIC:                         1.407e+04
Df Residuals:                    9998   BIC:                         1.408e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5674      0.051     30.955      0.000       1.468       1.667
x1            -1.1700      0.057    -20.544      0.000      -1.282      -1.058
==============================================================================
Omnibus:                    37825.818   Durbin-Watson:                   1.868
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1442.439
Skew:                          -0.082   Prob(JB):                         0.00
Kurtosis:                       1.147   Cond. No.                         20.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [428]:
# simple linear transformation of the model: at what estimated value of theta does the rw perform better on average
# Does not match with the estimated data!!!
(0.5 - res.params[0]) / res.params[1]

0.5505713975954748

In [429]:
## 2. BRUTE FORCE APPROACH: probably not the correct analysis but a good check

td = pd.DataFrame([theta_100,dm_100]).T
td.columns = ["Theta^", "DM res"]
td = td.sort_values("Theta^")
td.index = range(runs*10)

In [430]:
sampled_theta = []
sampled_dm = []

for i in range(int(runs/10)):
    start = i * 100
    end = (i + 1) * 100
    sampled_theta.append(td["Theta^"][start:end].mean())
    sampled_dm.append(td["DM res"][start:end].mean())

sampled_theta = np.asarray(sampled_theta)
sampled_dm = np.asarray(sampled_dm)

In [431]:
samp_td = pd.DataFrame([sampled_theta,sampled_dm]).T
samp_td.columns = ["Theta^", "DM res"]
samp_td[80:]

,Theta^,DM res
80,0.957014,0.12
81,0.958990,0.11
82,0.960874,0.09
83,0.962619,0.13
84,0.964517,0.06
85,0.966602,0.07
86,0.968634,0.06
87,0.970460,0.06
88,0.972596,0.08
89,0.974900,0.03


In [440]:
# 3. PROBIT MODEL

#y = ["Diebold"]
#x = sm.add_constant(ss_100["theta"])  x.astype(float)
model = Probit(dm_100, sm.add_constant(theta_100))
probit_model = model.fit()
probit_model.summary()

Optimization terminated successfully.
         Current function value: 0.669659
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                10000
Model:                         Probit   Df Residuals:                     9998
Method:                           MLE   Df Model:                            1
Date:                Fri, 12 Mar 2021   Pseudo R-squ.:                 0.03102
Time:                        09:32:56   Log-Likelihood:                -6696.6
converged:                       True   LL-Null:                       -6911.0
Covariance Type:            nonrobust   LLR p-value:                 2.995e-95
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9240      0.143     20.488      0.000       2.644       3.204
x1            -3.2036      0.160    -20.061      0.000      -3.517      -2.891
==============================================================================
"""

In [441]:
# Average marginal effect of theta on the probability at each observation
probit_model.get_margeff(at="overall", method='dydx', atexog=None, dummy=False, count=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                      y
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.2306      0.057    -21.426      0.000      -1.343      -1.118
==============================================================================
"""

In [442]:
# function expressing the probability of a dm rejection in terms of the estimated
norm.cdf(probit_model.params[0] + theta_100 * probit_model.params[1])

array([0.58762223, 0.48462291, 0.61536333, ..., 0.3912712 , 0.39830615,
       0.60986504])

In [443]:
# Value of theta at which it is better to approximate the distribution with a random walk
(norm.ppf(0.5) - probit_model.params[0]) / probit_model.params[1]

0.9127076166762589

In [423]:
np.max(probit_model.predict())

0.42530325956666104

In [335]:
np.mean(dm_100)

0.35579

QUESTION 2
--

In [101]:
# Simulate some data
# run time ~ 1 minute

# True simulated data ## Sample size 100
rg = np.random.RandomState(100)

r = [0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
runs = 1000
size = 100
nw = size ** (1/3)
cv = norm.ppf(0.975)
theta_100_2 = []
dm_100_2 = []

for rho in r:
    print(rho)
    for run in range(runs):
        start = rg.normal(0,1/(1-(rho**2)))
        run_holder = []
        run_holder.append(start)

        for i in range(1,size):
            shock = rg.standard_normal()
            last_val = run_holder[-1]
            new_val = last_val * rho + shock
            run_holder.append(new_val)
            
        y = np.asarray(run_holder)

        # Fit using first half, tau//2
        mod = AutoReg(y[:size//2], lags=1, trend="c", old_names=False)
        res = mod.fit()

        # Full-sample model
        oos_mod = AutoReg(y, lags=1, trend="c", old_names=False)

        # OOS Random Walk predictions
        oos_rw = oos_mod.predict([0, 1])

        ## ADF test, and forecasting
        adf = ADF(y[-size//2:]).pvalue

        if adf > 0.05:
            oos_1step = oos_rw
        else:
            oos_1step = oos_mod.predict(res.params)


        # Get second half of both
        # Use -size//2: to get second half
        oos_1step = oos_1step[-size//2:]
        oos_rw = oos_rw[-size//2:]

        # Append the estimated thetas and the DM results into lists
        theta_100_2.append(res.params[1])

        dm_100_2.append(diebold_mariano( oos_rw, oos_1step, nw, cv))


0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99


In [102]:
# Yes, pre-selecting our forecasts for a unit root improves out of sample forecasts a great deal

dm_100_2 = np.asarray(dm_100_2)
theta_100_2 = np.asarray(theta_100_2)
# x = np.asarray(pd.DataFrame([theta_100,(theta_100)**2]).T) ### can be used to get a non-linear relationship?
res = sm.OLS(dm_100_2, sm.add_constant(theta_100_2)).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     10.37
Date:                Fri, 12 Mar 2021   Prob (F-statistic):            0.00128
Time:                        06:55:57   Log-Likelihood:                 793.24
No. Observations:               10000   AIC:                            -1582.
Df Residuals:                    9998   BIC:                            -1568.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1270      0.023      5.485      0.000       0.082       0.172
x1            -0.0839      0.026     -3.220      0.001      -0.135      -0.033
==============================================================================
Omnibus:                     7928.184   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           107857.526
Skew:                           3.993   Prob(JB):                         0.00
Kurtosis:                      16.967   Cond. No.                         20.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""